In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
IMAGE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Datasets\Augment"
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 12

train = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
train = train.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

validation = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
validation = validation.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

Found 335500 files belonging to 12 classes.
Using 268400 files for training.
Found 335500 files belonging to 12 classes.
Using 67100 files for validation.


In [5]:
train = train.prefetch(buffer_size=32)
validation = validation.prefetch(buffer_size=32)

In [6]:
import h5py
import os, shutil
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from keras import layers, models, optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from squeezenet import SqueezeNet
from keras.callbacks import EarlyStopping

In [7]:
sn = SqueezeNet(input_shape=(256, 256, 3), nb_classes=NUM_CLASSES)
sn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 maxpool1 (MaxPooling2D)        (None, 63, 63, 64)   0           ['conv1[0][0]']                  
                                                                                              

                                                                                                  
 fire9_concatenate (Concatenate  (None, 15, 15, 512)  0          ['fire9_expand_1x1[0][0]',       
 )                                                                'fire9_expand_3x3[0][0]']       
                                                                                                  
 conv10 (Conv2D)                (None, 15, 15, 12)   6156        ['fire9_concatenate[0][0]']      
                                                                                                  
 avgpool10 (GlobalAveragePoolin  (None, 12)          0           ['conv10[0][0]']                 
 g2D)                                                                                             
                                                                                                  
 softmax (Activation)           (None, 12)           0           ['avgpool10[0][0]']              
          

In [8]:
sgd = SGD(lr=0.001, decay=0.0002, momentum=0.9, nesterov=True)
sn.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
CHECKPOINT_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\h5 Model\weights.h5"
EPOCHS = 300

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
checkpoint = ModelCheckpoint(                                         
    CHECKPOINT_PATH,
    monitor='val_loss',                               
    verbose=0,                                        
    save_best_only=True,                              
    save_weights_only=True,                           
    mode='min',                                       
    period=1
)

sn.fit_generator(
    train,
    validation_data=validation,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpoint]
)

Epoch 1/300
8388/8388 [==============================] - 295s 34ms/step - loss: 1.9448 - accuracy: 0.3241 - val_loss: 1.6100 - val_accuracy: 0.4485
Epoch 2/300
8388/8388 [==============================] - 277s 33ms/step - loss: 0.9109 - accuracy: 0.6956 - val_loss: 0.6348 - val_accuracy: 0.7949
Epoch 3/300
8388/8388 [==============================] - 277s 33ms/step - loss: 0.3306 - accuracy: 0.8969 - val_loss: 0.2304 - val_accuracy: 0.9286
Epoch 4/300
8388/8388 [==============================] - 274s 33ms/step - loss: 0.1455 - accuracy: 0.9562 - val_loss: 0.1767 - val_accuracy: 0.9471
Epoch 5/300
8388/8388 [==============================] - 276s 33ms/step - loss: 0.0790 - accuracy: 0.9765 - val_loss: 0.0551 - val_accuracy: 0.9834
Epoch 6/300
8388/8388 [==============================] - 279s 33ms/step - loss: 0.0482 - accuracy: 0.9856 - val_loss: 0.0583 - val_accuracy: 0.9823
Epoch 7/300
8388/8388 [==============================] - 277s 33ms/step - loss: 0.0330 - accuracy: 0.9902 - val_

KeyboardInterrupt: 

In [10]:
SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\h5 Model\squeezenet.h5"
sn.save(SAVE_PATH)

In [11]:
TFLITE_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\tflite Model\model.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(sn)
tfmodel = converter.convert()
open(TFLITE_SAVE_PATH, 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\offco\AppData\Local\Temp\tmpgoq6kr69\assets


INFO:tensorflow:Assets written to: C:\Users\offco\AppData\Local\Temp\tmpgoq6kr69\assets


2930736